### Creating Gifs (Debugging)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

##### Single Frame

In [2]:
def plot_keypoints(keypoints, title):
    # Separate pose, left hand, and right hand keypoints based on sizes
    pose = keypoints[:33 * 4].reshape(-1, 4)        # 33 keypoints, each with [x, y, z, visibility]
    left_hand = keypoints[33 * 4:33 * 4 + 21 * 3].reshape(-1, 3)  # 21 keypoints for left hand, each with [x, y, z]
    right_hand = keypoints[33 * 4 + 21 * 3:].reshape(-1, 3)       # 21 keypoints for right hand, each with [x, y, z]

    # Define connections (edges) between keypoints for pose, left hand, and right hand
    # These connections are based on a standard pose model, such as MediaPipe's Pose model.
    pose_connections = [
        (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
        (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19),
        (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20), (23, 24),
        (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
    ]

    hand_connections = [
        (0, 1), (1, 2), (2, 3), (3, 4), # Thumb
        (0, 5), (5, 6), (6, 7), (7, 8), # Index finger
        (0, 9), (9, 10), (10, 11), (11, 12), # Middle finger
        (0, 13), (13, 14), (14, 15), (15, 16), # Ring finger
        (0, 17), (17, 18), (18, 19), (19, 20) # Pinky
    ]

    # Plot keypoints with connections
    plt.figure(figsize=(4.8, 2.7))
    plt.axis([0, 1.125, 0, 2])

    # Plot pose keypoints and connections
    for (start, end) in pose_connections:
        plt.plot([pose[start, 0], pose[end, 0]], [pose[start, 1], pose[end, 1]], 'k-', lw=2)
    plt.scatter(pose[:, 0], pose[:, 1], label='Pose', s=20, alpha=0.7)

    # Plot left hand keypoints and connections
    for (start, end) in hand_connections:
        plt.plot([left_hand[start, 0], left_hand[end, 0]], [left_hand[start, 1], left_hand[end, 1]], 'b-', lw=2)
    plt.scatter(left_hand[:, 0], left_hand[:, 1], label='Left Hand', s=20, alpha=0.7)

    # Plot right hand keypoints and connections
    for (start, end) in hand_connections:
        plt.plot([right_hand[start, 0], right_hand[end, 0]], [right_hand[start, 1], right_hand[end, 1]], 'r-', lw=2)
    plt.scatter(right_hand[:, 0], right_hand[:, 1], label='Right Hand', s=20, alpha=0.7)

    # Adjust plot settings
    plt.title(title)
    plt.legend()
    # Adjust plot settings
    plt.legend()
    ax = plt.gca()
    leg = ax.get_legend()
    leg.legend_handles[0].set_color('black')
    leg.legend_handles[1].set_color('blue')
    leg.legend_handles[2].set_color('red')
    plt.axis('off')
    plt.gca().invert_yaxis()  # Invert Y axis for typical image coordinates
    plt.show()

In [3]:
def visualize_interpolation(gesture, video_num, frame_num):
    """
    Visualize the original and interpolated keypoints for a specific frame in a video.
    
    Parameters:
    gesture (str): Name of the gesture.
    video_num (int): Number of the video.
    frame_num (int): Number of the frame to visualize.
    """
    # Load original and interpolated keypoints
    npy_file = f'{frame_num}.npy'
    keypoints = np.load(os.path.join(video_directory, gesture, f'{video_num:02}', 'landmarks', npy_file))
    keypoints_interpolated = np.load(os.path.join(video_directory, gesture, f'{video_num:02}', 'interpolated_landmarks', npy_file))
    # keypoints_flipped = np.load(os.path.join(video_directory, gesture, f'{video_num:02}', 'flipped', npy_file))

    # Plot original and interpolated keypoints
    plot_keypoints(keypoints, title='Original Keypoints')
    plot_keypoints(keypoints_interpolated, title='Interpolated Keypoints')
    # plot_keypoints(keypoints_flipped, title='Flipped')

In [4]:
%%script false --no-raise-error

video_directory = r'temp'
visualize_interpolation('berapa', 28, 30)

Couldn't find program: 'false'


##### Gif Creation

In [2]:
def visualize_keypoints_to_image(keypoints):
    # Separate pose, left hand, and right hand keypoints based on sizes
    pose = keypoints[:33 * 4].reshape(-1, 4)        # 33 keypoints, each with [x, y, z, visibility]
    left_hand = keypoints[33 * 4:33 * 4 + 21 * 3].reshape(-1, 3)  # 21 keypoints for left hand, each with [x, y, z]
    right_hand = keypoints[33 * 4 + 21 * 3:].reshape(-1, 3)       # 21 keypoints for right hand, each with [x, y, z]

    # Define connections (edges) between keypoints for pose, left hand, and right hand
    # These connections are based on a standard pose model, such as MediaPipe's Pose model.
    pose_connections = [
        (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
        (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19),
        (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20), (23, 24),
        (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
    ]

    hand_connections = [
        (0, 1), (1, 2), (2, 3), (3, 4), # Thumb
        (0, 5), (5, 6), (6, 7), (7, 8), # Index finger
        (0, 9), (9, 10), (10, 11), (11, 12), # Middle finger
        (0, 13), (13, 14), (14, 15), (15, 16), # Ring finger
        (0, 17), (17, 18), (18, 19), (19, 20) # Pinky
    ]

        # Plot keypoints with connections
    fig, ax = plt.subplots(figsize=(19.2, 10.8))
    plt.axis([0, 1.125, 0, 2])

    # Plot pose keypoints and connections
    for (start, end) in pose_connections:
        plt.plot([pose[start, 0], pose[end, 0]], [pose[start, 1], pose[end, 1]], 'k-', lw=2)
    ax.scatter(pose[:, 0], pose[:, 1], label='Pose', s=20, alpha=0.7)

    # Plot left hand keypoints and connections
    for (start, end) in hand_connections:
        plt.plot([left_hand[start, 0], left_hand[end, 0]], [left_hand[start, 1], left_hand[end, 1]], 'b-', lw=2)
    ax.scatter(left_hand[:, 0], left_hand[:, 1], label='Left Hand', s=20, alpha=0.7)

    # Plot right hand keypoints and connections
    for (start, end) in hand_connections:
        plt.plot([right_hand[start, 0], right_hand[end, 0]], [right_hand[start, 1], right_hand[end, 1]], 'r-', lw=2)
    ax.scatter(right_hand[:, 0], right_hand[:, 1], label='Right Hand', s=20, alpha=0.7)

    # Adjust plot settings
    ax.legend()
    ax = plt.gca()
    leg = ax.get_legend()
    leg.legend_handles[0].set_color('black')
    leg.legend_handles[1].set_color('blue')
    leg.legend_handles[2].set_color('red')
    ax.axis('off')
    ax.invert_yaxis()  # Invert Y axis for typical image coordinates

    fig.canvas.draw()
    image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    plt.close(fig)

    return image

def create_gif_from_npy(input_folder, output_gif, duration=100):
    # Load all .npy files in sorted order
    npy_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.npy')])
    npy_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    images = []
    for npy_file in npy_files:
        # Load keypoints from each .npy file
        keypoints = np.load(os.path.join(input_folder, npy_file))
        
        # Visualize each frame as an image (without displaying the plot)
        image = visualize_keypoints_to_image(keypoints)
        
        # Add the image to the list of frames
        images.append(image)
    
    # Check if the output folder exists
    output_folder = os.path.dirname(output_gif)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Create and save the GIF
    images[0].save(
        output_gif,
        save_all=True,
        append_images=images[1:],
        duration=duration,
        loop=0
    )

In [4]:
# %%script false
folder = 'DATASET'
gesture = 'abang'
file = '01'
# create_gif_from_npy(os.path.join(folder, gesture, file, 'landmarks'), f'gif/{gesture}/{file}_landmarks.gif', duration=100)
create_gif_from_npy(os.path.join(folder, gesture, file, 'interpolated_landmarks'), f'gif/{gesture}/{file}_Err.gif', duration=100)
# create_gif_from_npy(os.path.join(folder, gesture, str(int(file) + 10)), f'gif/{gesture}/{file}_flipped.gif', duration=100)

C:\Users\MCYK\AppData\Local\Temp\ipykernel_21256\1186050422.py:54: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())


##### Normalized

In [7]:
def visualize_norm_keypoints_to_image(keypoints):
    # Separate pose, left hand, and right hand keypoints based on sizes
    pose = keypoints[:33 * 4].reshape(-1, 4)        # 33 keypoints, each with [x, y, z, visibility]
    left_hand = keypoints[33 * 4:33 * 4 + 21 * 3].reshape(-1, 3)  # 21 keypoints for left hand, each with [x, y, z]
    right_hand = keypoints[33 * 4 + 21 * 3:].reshape(-1, 3)       # 21 keypoints for right hand, each with [x, y, z]

    # Define connections (edges) between keypoints for pose, left hand, and right hand
    # These connections are based on a standard pose model, such as MediaPipe's Pose model.
    pose_connections = [
        (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
        (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19),
        (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20), (23, 24),
        (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
    ]

    hand_connections = [
        (0, 1), (1, 2), (2, 3), (3, 4), # Thumb
        (0, 5), (5, 6), (6, 7), (7, 8), # Index finger
        (0, 9), (9, 10), (10, 11), (11, 12), # Middle finger
        (0, 13), (13, 14), (14, 15), (15, 16), # Ring finger
        (0, 17), (17, 18), (18, 19), (19, 20) # Pinky
    ]

        # Plot keypoints with connections
    # fig, ax = plt.subplots(figsize=(19.2, 10.8))
    fig, ax = plt.subplots(figsize=(4.8, 2.7))
    plt.axis([-3, 3, -3, 3])

    # Plot pose keypoints and connections
    for (start, end) in pose_connections:
        plt.plot([pose[start, 0], pose[end, 0]], [pose[start, 1], pose[end, 1]], 'k-', lw=2)
    ax.scatter(pose[:, 0], pose[:, 1], label='Pose', s=20, alpha=0.7)

    # Plot left hand keypoints and connections
    for (start, end) in hand_connections:
        plt.plot([left_hand[start, 0], left_hand[end, 0]], [left_hand[start, 1], left_hand[end, 1]], 'b-', lw=2)
    ax.scatter(left_hand[:, 0], left_hand[:, 1], label='Left Hand', s=20, alpha=0.7)

    # Plot right hand keypoints and connections
    for (start, end) in hand_connections:
        plt.plot([right_hand[start, 0], right_hand[end, 0]], [right_hand[start, 1], right_hand[end, 1]], 'r-', lw=2)
    ax.scatter(right_hand[:, 0], right_hand[:, 1], label='Right Hand', s=20, alpha=0.7)

    # Adjust plot settings
    ax.legend()
    ax = plt.gca()
    leg = ax.get_legend()
    leg.legend_handles[0].set_color('black')
    leg.legend_handles[1].set_color('blue')
    leg.legend_handles[2].set_color('red')
    ax.axis('off')
    ax.invert_yaxis()  # Invert Y axis for typical image coordinates

    fig.canvas.draw()
    image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    plt.close(fig)

    return image

def create_norm_gif_from_npy(input_folder, output_gif, duration=100):
    # Load all .npy files in sorted order
    npy_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.npy')])
    npy_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    images = []
    for npy_file in npy_files:
        # Load keypoints from each .npy file
        keypoints = np.load(os.path.join(input_folder, npy_file))
        
        # Visualize each frame as an image (without displaying the plot)
        image = visualize_norm_keypoints_to_image(keypoints)
        
        # Add the image to the list of frames
        images.append(image)
    
    # Check if the output folder exists
    output_folder = os.path.dirname(output_gif)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Create and save the GIF
    images[0].save(
        output_gif,
        save_all=True,
        append_images=images[1:],
        duration=duration,
        loop=0
    )

In [8]:
%%script false
folder = 'TRAIN'
gesture = 'anak'
file = '13'
# create_norm_gif_from_npy(os.path.join(folder, gesture, str(int(file) + 10)), f'gif/{gesture}/{file}_flipped.gif', duration=100)
create_norm_gif_from_npy(os.path.join(folder, gesture, file), f'gif/{gesture}/{file}_normalized.gif', duration=100)

Couldn't find program: 'false'


In [9]:
video_directory = 'temp'

sum = 0

gesture_folder = np.array(os.listdir(video_directory))
for gestures in gesture_folder:
    gesture = []

    for fname in os.listdir(os.path.join(video_directory, gestures)):
        path = os.path.join(video_directory, gestures, fname)
        if os.path.isdir(path):
            gesture.append(fname)

    sum += len(gesture) 
    # print(gestures, end =" : ")        
    # print(len(gesture))

print("Total gestures: ", len(gesture_folder), "; Total videos: ", sum)

Total gestures:  24 ; Total videos:  1396


In [11]:
for gestures in gesture_folder:
    gesture = []

    for fname in os.listdir(os.path.join(video_directory, gestures)):
        path = os.path.join(video_directory, gestures, fname)
        if os.path.isdir(path):
            gesture.append(fname)
        
    # Iterate through half of the videos in each gesture only
    for no in gesture[:int(len(gesture)/2)]:
        load_path = os.path.join(video_directory, gestures, no)
        create_norm_gif_from_npy(load_path, f'gif/{gestures}/{no}_normalized.gif', duration=100)
        

C:\Users\MCYK\AppData\Local\Temp\ipykernel_15088\3171365109.py:55: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
